In [1]:
# Imports
import json, os, re, shutil, sys, time
import seaborn as sns
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML
from collections import defaultdict
import xmltodict
import untangle
import xml.etree.ElementTree as ET
# NLTK for NLP utils and corpora
import nltk
from nltk.corpus import treebank
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

# Import spacy
import spacy

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf

from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import minmax_scale

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.embeddings import Embedding

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

from sklearn.metrics import accuracy_score



Using TensorFlow backend.


In [2]:
aeg_long = pd.read_csv("../data-DNC/AEG/training_set_rel3.tsv",sep='\t',encoding = "latin1")
#aeg_long.columns = aeg_long.iloc[0]
#aeg_long.reindex(aeg_long.index.drop(0))
aeg_long.head(5)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
tree = ET.parse('../data-DNC/AESW/aesw2016_v1.2_train.xml')
root = tree.getroot()

In [4]:
sent_attrib = list()
sent_text = list()
for sent in root.iter('sentence'):
    sent_attrib.append(sent.attrib['sid'])
    sent_text.append(sent.text)

In [5]:
sent_dict = dict()
for sent in root.iter('sentence'):
    sid = sent.attrib['sid']
    sent_dict[sid]=sent.text

In [6]:
len(sent_dict)

1189412

### AESW Data for grammatical errors

In [7]:
aesw_train_labels = pd.read_csv("../data-DNC/AESW/aesw2016_v1.2_train.tok",sep='\t',encoding = "latin1",header=None)
aesw_train_labels.head(5)

,0,1,2
0,1,1.0,To facilitate an easier notation throughout t...
1,-1,1.0,To facilitate an easier notation throughout t...
2,0,1.1,Therefore MATH defines a special order of tim...
3,0,1.2,This is important since only MATH is the real...
4,0,1.3,Note that in all contour time-integrals we es...


In [8]:
aesw_train_labels.columns = ['Key_ID','Sent_ID','Text']
aesw_train_labels = aesw_train_labels[aesw_train_labels.Key_ID!=-1]
len(aesw_train_labels)

1196941

In [9]:
aesw_train_labels.head(5)

,Key_ID,Sent_ID,Text
0,1,1.0,To facilitate an easier notation throughout t...
2,0,1.1,Therefore MATH defines a special order of tim...
3,0,1.2,This is important since only MATH is the real...
4,0,1.3,Note that in all contour time-integrals we es...
5,0,2.0,Theorem REF proves the equivalence of ensembl...


In [10]:
x_train = aesw_train_labels.Text
y_train = aesw_train_labels.Key_ID

In [11]:
aesw_dev_labels = pd.read_csv("../data-DNC/AESW/aesw2016_v1.2_dev.tok",sep='\t',encoding = "latin1",header=None)
aesw_dev_labels.head(5)

,0,1,2
0,0,1.0,The team of robots in Chapter REF and CITE cr...
1,1,1.1,The Hamiltonian -LRB- Lyapunov function -RRB-...
2,-1,1.1,The Hamiltonian -LRB- Lyapunov function -RRB-...
3,1,1.2,"Therefore , for MATH robots , the time deriva..."
4,-1,1.2,"Therefore , for MATH robots the time derivati..."


In [12]:
aesw_dev_labels.columns = ['Key_ID','Sent_ID','Text']
aesw_dev_labels = aesw_dev_labels[aesw_dev_labels.Key_ID!=-1]
len(aesw_dev_labels)

148478

In [13]:
x_dev = aesw_dev_labels.Text
y_dev = aesw_dev_labels.Key_ID.as_matrix(columns=None)

In [14]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x_train)

In [15]:
sequences = tokenizer.texts_to_sequences(x_train)
data = pad_sequences(sequences, maxlen=50)

In [16]:
print(data.shape)

(1196941, 50)


In [17]:
dev_seq = tokenizer.texts_to_sequences(x_dev)
dev_data = pad_sequences(dev_seq, maxlen=50)
dev_data.shape

(148478, 50)

In [33]:
seed(2017)
conv = Sequential()
conv.add(Embedding(vocabulary_size, 100, input_length=50))
conv.add(Conv1D(64, 5, activation = 'relu'))
conv.add(MaxPooling1D(2))
conv.add(Flatten())
conv.add(Dense(1, activation = 'sigmoid'))
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
conv.fit(data, y_train, batch_size = 500, epochs = 40, verbose = 0)

KeyboardInterrupt: 

In [32]:
pred = conv.predict(dev_data)
pred_labels = (pred > 0.5).astype(np.int)
pred_labels = np.reshape(pred_labels,[148478,])

#print(y_dev[0:5])
#print(pred_labels[0:5])
accuracy = accuracy_score(y_dev,pred_labels)
print("Accuracy: ",accuracy)

Accuracy:  0.6545212085292097


In [27]:
print(np.shape(y_dev))
print(type(y_dev))
print(type(pred_labels))

(148478,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## TED

In [3]:
ted_main = pd.read_csv("../data-DNC/TED/ted_main.csv")
ted_transcript = pd.read_csv("../data-DNC/TED/transcripts.csv") 


In [3]:
ted_main.describe()

,comments,duration,film_date,languages,num_speaker,published_date,views
count,2550.000000,2550.000000,2.550000e+03,2550.000000,2550.000000,2.550000e+03,2.550000e+03
mean,191.562353,826.510196,1.321928e+09,27.326275,1.028235,1.343525e+09,1.698297e+06
std,282.315223,374.009138,1.197391e+08,9.563452,0.207705,9.464009e+07,2.498479e+06
min,2.000000,135.000000,7.464960e+07,0.000000,1.000000,1.151367e+09,5.044300e+04
25%,63.000000,577.000000,1.257466e+09,23.000000,1.000000,1.268463e+09,7.557928e+05
50%,118.000000,848.000000,1.333238e+09,28.000000,1.000000,1.340935e+09,1.124524e+06
75%,221.750000,1046.750000,1.412964e+09,33.000000,1.000000,1.423432e+09,1.700760e+06
max,6404.000000,5256.000000,1.503792e+09,72.000000,5.000000,1.506092e+09,4.722711e+07


In [4]:
ted_transcript.describe()

,transcript,url
count,2467,2467
unique,2464,2464
top,I'm going to tell you a little bit about my TE...,https://www.ted.com/talks/brene_brown_listenin...
freq,2,2


In [5]:
ted_main.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [6]:
ted_transcript.head(5)

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [17]:
print(len(ted_main))
print(len(ted_transcript))

2550
2467


### Not all TED MAIN has a transcript

In [24]:
ted_match = ted_main[ted_main.url.isin(ted_transcript.url)]
ted_notmatch = ted_main[~ted_main.url.isin(ted_transcript.url)]
len(ted_match)

2464

In [64]:
sentence = aeg_long.essay[0]
#sentence = ted_transcript.transcript[0]
s0 = nltk.word_tokenize(sentence)

In [65]:
t0 = Text(s0)

In [66]:
sentence_mod = ' '.join(s0)
print(sentence_mod)

Dear local newspaper , I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people , helps us learn about the globe ( astronomy ) and keeps us out of troble ! Thing about ! Dont you think so ? How would you feel if your teenager is always on the phone with friends ! Do you ever time to chat with your friends or buisness partner about things . Well now - there 's a new way to chat the computer , theirs plenty of sites on the internet to do so : @ ORGANIZATION1 , @ ORGANIZATION2 , @ CAPS1 , facebook , myspace ect . Just think now while your setting up meeting with your boss on the computer , your teenager is having fun on the phone not rushing to get off cause you want to use it . How did you learn about other countrys/states outside of yours ? Well I have by computer/internet , it 's a new way to learn about what going on in our time ! You might think your child spends a lot of time on the computer , but ask them s

In [67]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

lexical_diversity(t0)

0.4689119170984456

In [77]:
nlp = spacy.load('en_core_web_lg')

In [100]:
def most_similar(word):
    queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    return by_similarity[:4]

In [101]:
wordnet_lemmatizer = WordNetLemmatizer()
#word_syn_list = [wordnet_lemmatizer.lemmatize(w.lower_) for w in most_similar(nlp.vocab[u'Dear'])]
word_syn_list = [w.lower_ for w in most_similar(nlp.vocab[u'Safe'])]

In [102]:
set(word_syn_list)

{'safe', 'safety', 'secure'}

In [106]:
w_sample = nlp.vocab[u'Safe']
print(type(w_sample))
for w in w_sample.vocab:
    print(w.prob)

<class 'spacy.lexeme.Lexeme'>


In [68]:
doc = nlp(sentence_mod)

In [69]:
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in doc]

In [70]:
df_spacy = pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])
nltk_pos_tagged = nltk.pos_tag(sentence.split())
df_nltk = pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])


In [71]:
df_spacy

,Word,POS tag,Tag type
0,Dear,JJ,ADJ
1,local,JJ,ADJ
2,newspaper,NN,NOUN
3,",",",",PUNCT
4,I,PRP,PRON
5,think,VBP,VERB
6,effects,NNS,NOUN
7,computers,NNS,NOUN
8,have,VBP,VERB
9,on,IN,ADP


In [72]:
df_nltk

,Word,POS tag
0,Dear,NNP
1,local,JJ
2,"newspaper,",NN
3,I,PRP
4,think,VBP
5,effects,NNS
6,computers,NNS
7,have,VBP
8,on,IN
9,people,NNS


In [73]:
for entity in doc.ents:
    print(entity.text, entity.label_)

ORGANIZATION2 GPE
DATE1 ORG


In [111]:
stop_words = set(stopwords.words('english'))
word_syns = defaultdict(lambda: set())
for word_index,word in enumerate(s0):
    if word not in stop_words:
        word_pos = doc[word_index].pos_
        if word_pos == "ADJ":
            syns = wordnet.synsets(word,wordnet.ADJ)
        elif word_pos == "NOUN" or "PROPN":
            syns = wordnet.synsets(word,wordnet.NOUN)
        elif word_pos == "ADV":
            syns = wordnet.synsets(word,wordnet.ADV)
        else:
            continue
            #if word not in stop_words:
            #or word_pos == "NOUN" or word_pos == "VERB"
        for i in range(len(syns)):
            word_syns[word].add(syns[i].lemmas()[0].name())

print(word_syns)


defaultdict(<function <lambda> at 0x1a42ac9a60>, {'Dear': {'costly', 'beloved', 'dear'}, 'local': {'local', 'local_anesthetic'}, 'newspaper': {'newspaper'}, 'I': {'I', 'iodine', 'one'}, 'think': {'think'}, 'effects': {'effects', 'effect', 'consequence', 'impression'}, 'computers': {'computer', 'calculator'}, 'people': {'citizenry', 'people', 'multitude'}, 'great': {'bang-up', 'big', 'great', 'capital'}, 'learning': {'eruditeness', 'learning'}, 'give': {'give'}, 'us': {'U', 'United_States', 'uracil', 'uranium'}, 'time': {'clock_time', 'prison_term', 'time', 'meter', 'fourth_dimension'}, 'chat': {'New_World_chat', 'Old_World_chat', 'chat'}, 'helps': {'avail', 'assistant', 'aid'}, 'globe': {'Earth', 'globe', 'ball'}, 'astronomy': {'astronomy'}, 'keeps': {'support', 'keep', 'hold'}, 'Thing': {'matter', 'thing'}, 'feel': {'feel', 'spirit', 'tactile_property'}, 'teenager': {'adolescent'}, 'phone': {'telephone', 'phone', 'earphone'}, 'friends': {'ally', 'friend', 'Friend', 'acquaintance', 'su

In [112]:
# Use spacy instead of wordnet
word_syns_spacy = defaultdict(lambda: set())
for word_index,word in enumerate(s0):
    if word not in stop_words:
        word_pos = doc[word_index].pos_
        if word_pos == "ADJ" or word_pos == "NOUN" or word_pos == "PROPN" or word_pos == "ADV":
            word_syn_list = [w.lower_ for w in most_similar(nlp.vocab[word])]
            for syn_w in word_syn_list:
                word_syns_spacy[word].add(syn_w)

print(word_syns_spacy)

defaultdict(<function <lambda> at 0x1a42d03e18>, {'Dear': {'thank', 'dear'}, 'local': {'local', 'businesses', 'locally', 'regional'}, 'newspaper': {'tabloid', 'newspapers', 'magazine', 'newspaper'}, 'effects': {'effects', 'adverse', 'affect', 'effect'}, 'computers': {'laptops', 'computers', 'desktops', 'computer'}, 'people': {'individuals', 'those', 'people', 'others'}, 'great': {'fantastic', 'good', 'great', 'wonderful'}, 'learning': {'learn', 'teach', 'teaching', 'learning'}, 'skills/affects': {'educated', 'wade', 'eradicate', 'equator'}, 'chat': {'chatroom', 'chatting', 'chat', 'chats'}, ',': {'and', ',', ';'}, 'us': {'our', 'us', 'we', 'them'}, 'learn': {'understand', 'teach', 'learn', 'learning'}, '!': {':)', '!', ':-)', ';)'}, 'How': {'what', 'how'}, 'feel': {'felt', 'feels', 'feel', 'feeling'}, 'phone': {'phone', 'cellphone', 'phones', 'telephone'}, 'friends': {'buddies', 'pals', 'friend', 'friends'}, 'time': {'day', 'time', 'once', 'when'}, 'things': {'things', 'everything', 's

In [110]:
wordnet.synsets('safe',wordnet.NOUN)

[Synset('safe.n.01'), Synset('safe.n.02'), Synset('condom.n.01')]